Importando as bibliotecas

In [2]:
#Selenium - Webscrapping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
#Biblioteca para implementar pausas no script
import time
#Pandas - Manipulação de dados
import pandas as pd

Definindo a cidades e negócios

In [7]:
cidades = ['Uruguaiana','Erechim','Cachoeiro de Itapemirim','Colatina',
           'Palmas','Araguaína', 'Açailândia','Balsas','Passo Fundo','Guarapuava','Bauru',
           'Patos de Minas','Barbacena','Varginha','Teófilo Otoni','Tangará da Serra','Santarém','Parauapebas',
          'Castanhal','Altamira','Lucas do Rio Verde','Florianópolis','Chapecó','Tubarão','Barretos','Ijuí','Campo Mourão','Caratinga', 'Manhuaçu',
          'Ponta Grossa','Foz do Iguaçu','Araucária','Arapongas',
          'São José dos Campos','Franca','Marilia','Bragança Paulista','Pouso Alegre','São José do Rio Preto','São Carlos',
           'Guaratinguetá','Juiz de Fora','Macaé', 'Sete Lagoas','Montes Claros','Pelotas','Vitória da Conquista','Ilhéus',
          'Cascavel','Marabá','Governador Valadares','Muriaé','Volta Redonda','Uberaba','Criciúma',
            'Capelinha','Guanhães','Conceição do Mato Dentro','Cataguases','Patrocínio','Janaúba',
           'Almenara','Formiga','Arcos','Bom Despacho','Lagoa da Prata','Bambuí',
           'Araçuaí','Arcoverde','Serra Talhada','Salgueiro','Ouricuri','Araripina',
           'Pedro Leopoldo','Confins','Curvelo','Itabirito','Ituiutaba','Itumbiara','Jequitinhonha',
          'Três Pontas','Nepomuceno','Conselheiro Lafaiete','Itabira','João Monlevade','Itaberaí','Itapuranga',
          'Cianorte','Telêmaco Borba','Guarapuava','Imperatriz','Três Lagoas','Sinop','Caldas Novas','Divinópolis','Ipatinga',
          'Três Marias','Jaboticabal','Votuporanga','Ubá','Marataízes',
            'Itapetinga','Jequié','Santo Antônio de Jesus',
          'Feira de Santana','Alagoinhas','Francisco Beltrão','Pato Branco','Alegrete','Gramado','Bagé','São Borja',
          'Carangola','Paragominas','Goianésia','Caxambu',
          'Ipatinga','Dourados','Três Lagoas','Corumbá','Ponta Porã',
          'Caruaru','Garanhuns','Santa Cruz do Capibaribe']


estabelecimentos = ['Posto de gasolina']
gabarito = ['Posto de combustível']

# estabelecimentos = ['Depósito de construção']
# gabarito = ['Loja de materiais de construção']

# estabelecimentos = ['Pet Shop']
# gabarito = ['Pet Shop']


In [18]:
#Conferindo coletas apontadas como suspeitas após o tratamento dos dados
cidades = ['Bragança Paulista','Nepomuceno','Araucária']
cidades = ['Patrocínio','Varginha','Patos de Minas']
cidades = ['Cascavel','Palmas','Carangola']
cidades = ['Varginha']

estabelecimentos = ['Depósito de construção']
gabarito = ['Loja de materiais de construção']



In [10]:
#Cidades usadas para validar o modelo
cidades = ['Araraquara', 'Eunápolis', 
            'Paragominas', 'Araguari', 'Itapetininga','Araxá','Cruz Alta','Cáceres']
estabelecimentos = ['Posto de gasolina']
gabarito = ['Posto de combustível']
# estabelecimentos = ['Pet Shop']
# gabarito = ['Pet Shop']
# estabelecimentos = ['Depósito de construção']
# gabarito = ['Loja de materiais de construção']

In [11]:
df = pd.DataFrame(columns = ['Cidade','Negócio','Contagem'])

Efetuando a extração 

In [12]:
# Iniciando o navegador automático
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

In [13]:
for cidade in cidades:
    print(df)
    for estab in estabelecimentos:
        url = 'https://www.google.com/maps/search/'+ estab + ' em ' + cidade
        driver.get(url)
        quantidade = 0
        botaoProximo = WebDriverWait(driver,7).until(EC.visibility_of_element_located((By.ID,'ppdPk-Ej1Yeb-LgbsSe-tJiF1e')))
        nNegocios = 0
        rodada = 1
        #Chegando até a última página
        b = 2
        while  b > 1:
            time.sleep(4)
            elementosResultados = WebDriverWait(driver,7).until(EC.presence_of_element_located((By.CLASS_NAME, "Jl2AFb")))
            if (elementosResultados.text == ''):
                break
            quantidade = int(elementosResultados.text.rsplit('-')[1].strip())
            if rodada > 1:
                quantidade = quantidade - 20*(rodada-1)
            # Contando os negócios nessa página
            for b in range(1,quantidade*2,2):
                print(b)
                a = driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[4]/div[1]/div['+str(b)+']/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]')
                a.location_once_scrolled_into_view
                tipoNegocio = a.text
                if tipoNegocio in gabarito:
                    nNegocios = nNegocios + 1
                time.sleep(0.2)
            rodada = rodada + 1
            if botaoProximo.get_property('disabled') == True:
                break
            try:    
                botaoProximo.click()
            except (ElementClickInterceptedException,TimeoutException, ElementNotInteractableException):
                print('Deu ruim',cidade, estab)
                break
            if quantidade%20 != 0:
                break
        lista = [cidade, estab, nNegocios]
        dfUnitario = pd.DataFrame(data = {'Cidade': [lista[0]],'Negócio': [lista[1]], 'Contagem':[lista[2]]})
        df = df.append([dfUnitario])

Empty DataFrame
Columns: [Cidade, Negócio, Contagem]
Index: []
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
       Cidade            Negócio Contagem
0  Araraquara  Posto de gasolina      106
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
       Cidade            Negócio Contagem
0  Araraquara  Posto de gasolina      106
0   Eunápolis  Posto de gasolina       34
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37


KeyboardInterrupt: 

In [14]:
df

,Cidade,Negócio,Contagem
0,Araraquara,Posto de gasolina,106
0,Eunápolis,Posto de gasolina,34


In [39]:
    #Extrações para construir o modelo
df.to_excel('A_Postos de Gasolina')
# df.to_excel('B_Depósitos de Construção')
# df.to_excel('C_Petshops')

    #Extrações voltadas para validação
df.to_excel('AV_Postos.xlsx')
# df.to_excel('AV_Depositos.xlsx')
# df.to_excel('AV_Petshops.xlsx')